## Techniques, Principles and Algorithms that go into building collaborative filters

### Similarity Measures
How can we mathematically quantify how different or similar two items are to each other

 - Euclidean distance - the length of the line segment joining two data points plotted on an n-dimensional Cartesian plane **The lower the Euclidean score, the more similar the two points are to each other**

In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# function to compute Euclidean distance
def euclidean(v1, v2):
    # convert 1-D Python lists to numpy vectors
    v1 = np.array(v1)
    v2 = np.array(v2)
    # compute vectors which is the element wise square of the distance
    diff = np.power(np.array(v1) - np.array(v2), 2)
    # perform summation of the elements of the above vector
    sigma_val = np.sum(diff)
    # compute square root amd return final euclidean score
    euclid_score =np.sqrt(sigma_val)
    return euclid_score